In [1]:
import sqlparse

from django.db import connection, reset_queries

In [2]:
post = Post.objects.get(slug="november-2023-11-13")

In [3]:
def count_queries(func):
    reset_queries()
    renditions = list(func())
    return len(connection.queries)


def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)    


#count_queries(post.get_all_renditions)

In [4]:
%%time
reset_queries()
renditions = list(post.get_all_renditions())
len(connection.queries)

CPU times: user 10 ms, sys: 2.71 ms, total: 12.7 ms
Wall time: 18.8 ms


1

In [5]:
a, b = renditions[0], renditions[1]

In [6]:
a == b

False

In [7]:
a

<Rendition: Rendition object (30465)>

In [8]:
b

<Rendition: Rendition object (30466)>

## Get all renditions for regular images in one query

In [32]:
print(Rendition.objects.filter(image__in=post.images.all()))

<QuerySet [<Rendition: Rendition object (30465)>, <Rendition: Rendition object (30514)>, <Rendition: Rendition object (30515)>, <Rendition: Rendition object (30516)>, <Rendition: Rendition object (30517)>, <Rendition: Rendition object (30518)>]>


In [44]:
reset_queries()
renditions = list(Rendition.objects.filter(image__in=post.images.all()))
print(len(connection.queries))
show_queries(connection.queries)

1
SELECT "wagtailimages_rendition"."id",
       "wagtailimages_rendition"."filter_spec",
       "wagtailimages_rendition"."file",
       "wagtailimages_rendition"."width",
       "wagtailimages_rendition"."height",
       "wagtailimages_rendition"."focal_point_key",
       "wagtailimages_rendition"."image_id"
FROM "wagtailimages_rendition"
WHERE "wagtailimages_rendition"."image_id" IN
    (SELECT U0."id"
     FROM "wagtailimages_image" U0
     INNER JOIN "cast_post_images" U1 ON (U0."id" = U1."image_id")
     WHERE U1."post_id" = 480)


In [45]:
len(renditions)

6

## Get all renditions for gallery images in one query

In [36]:
gallery = Gallery.objects.first()

In [38]:
Image.objects.filter(gallery__in=post.galleries.all())

<ImageQuerySet [<Image: Claas, Leander und Carl klettern immer noch auf der Kletterspinne>, <Image: Claas, Leander und Carl klettern auf einer Kletterspinne>, <Image: Nora Claas und das Neinhorn nochmal>, <Image: Nora und Claas und das Neinhorn>, <Image: Puppentheater am Staufenplatz>, <Image: Martinsfeier in der Schule>, <Image: Nora mit Laterne auf dem Schulhof>, <Image: Claas Klasse geht zur Martinsfeier in der Schule>, <Image: Nora mit Jasmin auf der Bank>, <Image: Der Kita-Martinsumzug geht jetzt wirklich gleich los>, <Image: Der Kita-Martinsumzug geht gleich los>, <Image: Nora mit Laterne im Kinderwagen>]>

In [39]:
print(Rendition.objects.filter(image__in=Image.objects.filter(gallery__in=post.galleries.all())))

<QuerySet [<Rendition: Rendition object (30466)>, <Rendition: Rendition object (30467)>, <Rendition: Rendition object (30468)>, <Rendition: Rendition object (30469)>, <Rendition: Rendition object (30470)>, <Rendition: Rendition object (30471)>, <Rendition: Rendition object (30472)>, <Rendition: Rendition object (30473)>, <Rendition: Rendition object (30474)>, <Rendition: Rendition object (30475)>, <Rendition: Rendition object (30476)>, <Rendition: Rendition object (30477)>, <Rendition: Rendition object (30531)>, <Rendition: Rendition object (30532)>, <Rendition: Rendition object (30533)>, <Rendition: Rendition object (30534)>, <Rendition: Rendition object (30535)>, <Rendition: Rendition object (30536)>, <Rendition: Rendition object (30537)>, <Rendition: Rendition object (30538)>, '...(remaining elements truncated)...']>


In [40]:
reset_queries()
renditions = list(Rendition.objects.filter(image__in=Image.objects.filter(gallery__in=post.galleries.all())))
print(len(connection.queries))
show_queries(connection.queries)

1
SELECT "wagtailimages_rendition"."id",
       "wagtailimages_rendition"."filter_spec",
       "wagtailimages_rendition"."file",
       "wagtailimages_rendition"."width",
       "wagtailimages_rendition"."height",
       "wagtailimages_rendition"."focal_point_key",
       "wagtailimages_rendition"."image_id"
FROM "wagtailimages_rendition"
WHERE "wagtailimages_rendition"."image_id" IN
    (SELECT V0."id"
     FROM "wagtailimages_image" V0
     INNER JOIN "cast_gallery_images" V1 ON (V0."id" = V1."image_id")
     WHERE V1."gallery_id" IN
         (SELECT U0."id"
          FROM "cast_gallery" U0
          INNER JOIN "cast_post_galleries" U1 ON (U0."id" = U1."gallery_id")
          WHERE U1."post_id" = 480))


In [42]:
len(list(post.get_all_renditions()))

150

In [43]:
len(renditions)

144

In [56]:
post.images.all().values_list("pk", flat=True)

<ImageQuerySet [2085]>

In [57]:
Image.objects.filter(gallery__in=post.galleries.all()).union(post.images.all()).values_list("pk", flat=True)

<ImageQuerySet [2089, 2083, 2079, 2085, 2078, 2084, 2080, 2086, 2082, 2077, 2088, 2087, 2081]>

## Get all renditions in one go!

In [61]:
Rendition.objects.filter(image__in=Image.objects.filter(gallery__in=post.galleries.all()).union(post.images.all()).values_list("pk", flat=True))

<QuerySet [<Rendition: Rendition object (30465)>, <Rendition: Rendition object (30466)>, <Rendition: Rendition object (30467)>, <Rendition: Rendition object (30468)>, <Rendition: Rendition object (30469)>, <Rendition: Rendition object (30470)>, <Rendition: Rendition object (30471)>, <Rendition: Rendition object (30472)>, <Rendition: Rendition object (30473)>, <Rendition: Rendition object (30474)>, <Rendition: Rendition object (30475)>, <Rendition: Rendition object (30476)>, <Rendition: Rendition object (30477)>, <Rendition: Rendition object (30478)>, <Rendition: Rendition object (30479)>, <Rendition: Rendition object (30480)>, <Rendition: Rendition object (30481)>, <Rendition: Rendition object (30482)>, <Rendition: Rendition object (30483)>, <Rendition: Rendition object (30484)>, '...(remaining elements truncated)...']>

In [63]:
%%time
reset_queries()
renditions = list(Rendition.objects.filter(image__in=Image.objects.filter(gallery__in=post.galleries.all()).union(post.images.all()).values_list("pk", flat=True)))
print(len(connection.queries))
show_queries(connection.queries)

1
SELECT "wagtailimages_rendition"."id",
       "wagtailimages_rendition"."filter_spec",
       "wagtailimages_rendition"."file",
       "wagtailimages_rendition"."width",
       "wagtailimages_rendition"."height",
       "wagtailimages_rendition"."focal_point_key",
       "wagtailimages_rendition"."image_id"
FROM "wagtailimages_rendition"
WHERE "wagtailimages_rendition"."image_id" IN (
                                                 (SELECT V0."id" AS "col1"
                                                  FROM "wagtailimages_image" V0
                                                  INNER JOIN "cast_gallery_images" V1 ON (V0."id" = V1."image_id")
                                                  WHERE V1."gallery_id" IN
                                                      (SELECT U0."id"
                                                       FROM "cast_gallery" U0
                                                       INNER JOIN "cast_post_galleries" U1 ON (U0."id" = U1."gallery_

## Using wagtails prefetch_renditions

Probably does not work...

In [67]:
post.images.all().prefetch_renditions()

<ImageQuerySet [<Image: Nora beim Kinderarzt>]>

In [68]:
# Image.objects.filter(gallery__in=post.galleries.all()).union(post.images.all()).prefetch_renditions()